In [45]:
import os
import sys
import logging
import uuid
import kfp

from datetime import datetime
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google import experimental
from kfp.v2.google.client import AIPlatformClient

sys.path.append('pipelines')
from pipelines.pipeline import taxi_tip_predictor_pipeline

In [46]:
kfp.__version__

'1.6.3'

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
PROJECT = 'jk-mlops-dev'
STAGING_BUCKET = 'gs://jk-vertex-workshop-bucket'
REGION = 'us-central1'
PIPELINES_SA = 'pipelines-sa@jk-mlops-dev.iam.gserviceaccount.com'


TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
TIMESTAMP

In [ ]:
@component
def training_op(
    project: str,
    region: str,
    staging_bucket: str,
    service_account: str,training_table: str,
    validation_table: str,
    epochs: int,
    per_replica_batch_size: int =128,
    machine_type: str = 'n1-standard-4',
    'accelerator_typ': 'NVIstr = DIA_TESLA_T4'
    accelerator_count': in = 1):
    
from google.cloud import aiplatform as vertex_ai
          vertex_ai.init(
        project=project,
        location=region,
    )   )dir    
    





In [79]:
VERTEX_TRAINING_JOB_NAME = 'taxi-tip-predictor-training-job'
PIPELINE_NAME = 'taxi-tip-predictor-continuous-training'

@kfp.dsl.pipeline(name=PIPELINE_NAME)
def taxi_tip_predictor_pipeline(
    project: str,
    model_display_name: str,
    training_container: str,
    epochs: int,
    per_replica_batch_size: int,
    training_table: str = 'jk-mlops-dev.chicago_taxi_training.training_split',
    validation_table: str = 'jk-mlops-dev.chicago_taxi_training.validation_split',
    machine_type: str = 'n1-standard-4',
    accelerator_type: str = 'NVIDIA_TESLA_T4',
    accelerator_count: str = '1',
    replica_count: str = '1', 
):

    
    

### Compile the pipeline

In [80]:
package_path = 'taxi_tip_predictor_pipeline.json'
compiler.Compiler().compile(
    pipeline_func=taxi_tip_predictor_pipeline,
    package_path=package_path
)

### Submit a pipeline run

In [81]:
api_client = AIPlatformClient(
    project_id=PROJECT,
    region=REGION,
)

In [82]:
pipeline_root = f'{STAGING_BUCKET}/pipelines'
model_display_name = 'Taxi tip predictor'
training_container_image = 'gcr.io/jk-mlops-dev/taxi_classifier_trainer'
epochs = 3
per_replica_batch_size = 128
training_table = 'jk-mlops-dev.chicago_taxi_training.training_split'
validation_table = 'jk-mlops-dev.chicago_taxi_training.validation_split'


parameter_values = {
    'project': PROJECT,
    'model_display_name': model_display_name,
    'training_container': training_container_image,
    'epochs': epochs,
    'per_replica_batch_size': per_replica_batch_size,
    'training_table': training_table,
    'validation_table': validation_table,
}

response = api_client.create_run_from_job_spec(
    package_path,
    pipeline_root=pipeline_root,
    parameter_values=parameter_values,
    service_account=PIPELINES_SA
)